In [2]:
import xarray as xr
import glob
import pandas as pd
import os
# from collections import defaultdict
# import re

In [3]:
ds = xr.open_dataset('/gws/nopw/j04/eo_shared_data_vol2/scratch/pete_nut/shipping_ccf_combined_dataset.nc')

In [4]:
ds

<xarray.Dataset>
Dimensions:                               (latitude: 21, longitude: 23,
                                           time: 2499)
Coordinates:
  * latitude                              (latitude) int64 -50 -45 -40 ... 45 50
  * longitude                             (longitude) int64 -90 -85 ... 15 20
  * time                                  (time) datetime64[ns] 2014-01-01 .....
Data variables: (12/63)
    10m_u_component_of_wind               (latitude, longitude, time) float64 ...
    10m_v_component_of_wind               (latitude, longitude, time) float64 ...
    COT_1                                 (latitude, longitude, time) float64 ...
    COT                                   (latitude, longitude, time) float64 ...
    COT_3                                 (latitude, longitude, time) float64 ...
    EIS                                   (latitude, longitude, time) float64 ...
    ...                                    ...
    v_component_of_wind_700hPa            (latitude, longitude, time) float64 ...
    v_component_of_wind_925hPa            (latitude, longitude, time) float64 ...
    vertical_velocity_300hPa              (latitude, longitude, time) float64 ...
    vertical_velocity_500hPa              (latitude, longitude, time) float64 ...
    vertical_velocity_700hPa              (latitude, longitude, time) float64 ...
    vertical_velocity_925hPa              (latitude, longitude, time) float64 ...